In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
import datetime

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[43491]: Class CaptureDelegate is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x147476480) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x11d590860). One of the two will be used. Which one is undefined.
objc[43491]: Class CVWindow is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x1474764d0) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x10ea04a68). One of the two will be used. Which one is undefined.
objc[43491]: Class CVView is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x1474764f8) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packa

### 1. Set up important functions

In [17]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame from OpenCV to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


def calculate_angle(point1: list, point2: list, point3: list) -> float:
    '''
    Calculate the angle between 3 points
    Unit of the angle will be in Degree
    '''
    point1 = np.array(point1)
    point2 = np.array(point2)
    point3 = np.array(point3)

    # Calculate algo
    angleInRad = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angleInDeg = np.abs(angleInRad * 180.0 / np.pi)

    angleInDeg = angleInDeg if angleInDeg <= 180 else 360 - angleInDeg
    return angleInDeg


def save_frame_as_image(frame, message: str = None):
    '''
    Save a frame as image to display the error
    '''
    now = datetime.datetime.now()

    if message:
        cv2.putText(frame, message, (50, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 3, cv2.LINE_AA)
        
    print("Saving ...")
    cv2.imwrite(f"../data/logs/bicep_{now}.jpg", frame)

### 2. Counter, Errors detection

In [5]:
VIDEO_PATH = "../data/db_curl/db_curl_1.mov"
VIDEO_PATH2 = "../data/db_curl/db_curl_bad_1.mov"

In [19]:
cap = cv2.VideoCapture(VIDEO_PATH2)

# Params for counter
STAGE_UP_THRESHOLD = 90
STAGE_DOWN_THRESHOLD = 120
stage = "down"
counter = 0

# Params to catch FULL RANGE OF MOTION error
PEAK_CONTRACTION_THRESHOLD = 70
peak_contraction_degree = 1000
peaked_error_framed = None

# LOOSE UPPER ARM error detection
LOOSE_UPPER_ARM = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Reduce size of a frame
        image = rescale_frame(image, 50)
        
        video_dimensions = [image.shape[1], image.shape[0]]

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

        # Make detection
        try:
            landmarks = results.pose_landmarks.landmark

            # Get joints' coordinates
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

            # Calculate curl angle for counter
            angle = int(calculate_angle(shoulder, elbow, wrist))
            if angle > STAGE_DOWN_THRESHOLD:
                stage = "down"
            elif angle < STAGE_UP_THRESHOLD and stage == "down":
                stage = "up"
                counter += 1
            
            # Calculate the angle between the upper arm (shoulder & joint) and the Y axis
            shoulder_projection = [ shoulder[0], 1 ] # Represent the projection of the shoulder to the X axis
            ground_upper_arm_angle = int(calculate_angle(elbow, shoulder, shoulder_projection))

            # * Evaluation for LOOSE UPPER ARM error
            if ground_upper_arm_angle > 40:
                # Limit the saved frame
                if not LOOSE_UPPER_ARM:
                    LOOSE_UPPER_ARM = True
                    save_frame_as_image(image, "Loose upper arm")
                else:
                    pass
            else:
                LOOSE_UPPER_ARM = False
            

            # Evaluate PEAK CONTRACTION error
            if stage == "up" and angle < peak_contraction_degree:
                # Capture peaked contraction every rep
                peak_contraction_degree = angle
                peaked_error_framed = image
                
            elif stage == "down":
                # Evaluate if the peak is higher than the threshold than capture that image and marked as an error
                if peak_contraction_degree != 1000 and peak_contraction_degree >= PEAK_CONTRACTION_THRESHOLD:
                    print(peak_contraction_degree)
                    save_frame_as_image(peaked_error_framed, "Peak contraction could be better")
                
                # Reset params
                peak_contraction_degree = 1000
                peaked_framed = None
            
            # Visualize calculated angles
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, video_dimensions).astype(int)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(ground_upper_arm_angle), tuple(np.multiply(shoulder, video_dimensions).astype(int)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Visualize Counter
            cv2.putText(image, stage + " " + str(counter), (50, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        except Exception as e:
            print(f"Error: {e}")
        
        cv2.imshow("CV2", image)
        
        # Press Q to close cv2 window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)
  

Saving ...
Saving ...
Saving ...
